# Spark Dataframes

* Look like pandas dataframes
* share some of the same methods and syntax
* but they are 2 separate types of objects

### 1. Create Spark Session

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

### 2. Crate Dataframes

Convert pandas dataframe into a spark dataframe

In [2]:
# Create pandas dataframe

import pandas as pd
import numpy as np


np.random.seed(456)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


### 3. Create Spark Dataframe from Pandas Dataframe

In [6]:
sp_df = spark.createDataFrame(pandas_dataframe)
sp_df

DataFrame[n: bigint, group: string]

* We do see the column names, but don't see the data
* Because spark is lazy, in that it won't show us values until it has to
* To peek, use `.show`
    * `.show` defaults to 20

In [8]:
sp_df.show()

+---+-----+
|  n|group|
+---+-----+
|  0|    b|
|  1|    b|
|  2|    c|
|  3|    a|
|  4|    c|
|  5|    c|
|  6|    a|
|  7|    b|
|  8|    a|
|  9|    b|
| 10|    b|
| 11|    a|
| 12|    b|
| 13|    a|
| 14|    b|
| 15|    b|
| 16|    c|
| 17|    c|
| 18|    a|
| 19|    c|
+---+-----+



### 4. Read Data from files

In [9]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



## Columns

* The following will create a series from a pandas dataframe, but a column object from a spark dataframe. 
* A column object represents a vertical slice of a dataframe, but does not contain the data itself. 
* You will use it to perform function on and reference that column

In [10]:
mpg.year

Column<b'year'>

In [16]:
mpg.select(mpg.hwy, mpg.cty, mpg.model).show(10)

+---+---+----------+
|hwy|cty|     model|
+---+---+----------+
| 29| 18|        a4|
| 29| 21|        a4|
| 31| 20|        a4|
| 30| 21|        a4|
| 26| 16|        a4|
| 26| 18|        a4|
| 27| 18|        a4|
| 26| 18|a4 quattro|
| 25| 16|a4 quattro|
| 28| 20|a4 quattro|
+---+---+----------+
only showing top 10 rows



In [17]:
# Column objects support operations such as arithmetic operations

mpg.hwy + 1

Column<b'(hwy + 1)'>

In [19]:
mpg.select(mpg.hwy, mpg.hwy + 1).show(2)

+---+---------+
|hwy|(hwy + 1)|
+---+---------+
| 29|       30|
| 29|       30|
+---+---------+
only showing top 2 rows



In [21]:
# We can rename the columns as an alias

mpg.select(mpg.hwy.alias("highway_mileage"),
           (mpg.hwy + 1).alias("hwy_miledge_plus1")).show()

+---------------+-----------------+
|highway_mileage|hwy_miledge_plus1|
+---------------+-----------------+
|             29|               30|
|             29|               30|
|             31|               32|
|             30|               31|
|             26|               27|
|             26|               27|
|             27|               28|
|             26|               27|
|             25|               26|
|             28|               29|
|             27|               28|
|             25|               26|
|             25|               26|
|             25|               26|
|             25|               26|
|             24|               25|
|             25|               26|
|             23|               24|
|             20|               21|
|             15|               16|
+---------------+-----------------+
only showing top 20 rows



In [23]:
# We can also store column objects in variables and reference them

col1 = mpg.hwy.alias("highway_mileage")
col2 = (mpg.hwy / 2).alias("highway_mileage_halved")
mpg.select(col1, col2).show()

+---------------+----------------------+
|highway_mileage|highway_mileage_halved|
+---------------+----------------------+
|             29|                  14.5|
|             29|                  14.5|
|             31|                  15.5|
|             30|                  15.0|
|             26|                  13.0|
|             26|                  13.0|
|             27|                  13.5|
|             26|                  13.0|
|             25|                  12.5|
|             28|                  14.0|
|             27|                  13.5|
|             25|                  12.5|
|             25|                  12.5|
|             25|                  12.5|
|             25|                  12.5|
|             24|                  12.0|
|             25|                  12.5|
|             23|                  11.5|
|             20|                  10.0|
|             15|                   7.5|
+---------------+----------------------+
only showing top

In addition to the syntax we've seen above, we can create columns with the `col` and `expr` functions from `pyspark.sql.function` module. 

In [25]:
from pyspark.sql.functions import col, expr
col("hwy")
# above is same as this  mpg.hwy

Column<b'hwy'>

In [26]:
avg_column = (col("hwy") + col("cty")) / 2

mpg.select(
    col("hwy").alias("highway_mileage"),
    mpg.cty.alias("city_mileage"),
    avg_column.alias("avg_mileage"),
).show(5)

+---------------+------------+-----------+
|highway_mileage|city_mileage|avg_mileage|
+---------------+------------+-----------+
|             29|          18|       23.5|
|             29|          21|       25.0|
|             31|          20|       25.5|
|             30|          21|       25.5|
|             26|          16|       21.0|
+---------------+------------+-----------+
only showing top 5 rows



Here we create a variable named avg_column that represents the average of the highway and city mileage of each vehicle. This variable is created by using the col function to produce pyspark Column objects and using the arithmetic operators to combine them.

Next we select the original highway and city mileage columns, in addition to our new average mileage column. We demonstrate the col function to select the hwy column and refer to the city mileage column with the df.cty syntax we saw previously. We also give all of our columns more readable aliases before showing the resulting dataframe.

expr

The expr function is more powerful than col. It does everything col does and more. expr returns the same type of column object, but allows us to express manipulations to the column within the string that defines the column.

In [27]:
mpg.select(
    expr("hwy"),  # the same as `col`
    expr("hwy + 1"),  # an arithmetic expression
    expr("hwy AS highway_mileage"),  # using an alias
    expr("hwy + 1 AS highway_incremented"),  # a combination of the above
).show(5)

+---+---------+---------------+-------------------+
|hwy|(hwy + 1)|highway_mileage|highway_incremented|
+---+---------+---------------+-------------------+
| 29|       30|             29|                 30|
| 29|       30|             29|                 30|
| 31|       32|             31|                 32|
| 30|       31|             30|                 31|
| 26|       27|             26|                 27|
+---+---------+---------------+-------------------+
only showing top 5 rows



In [28]:
#Note that all the columns created below are identical,
# and which syntax to use is merely a style choice.
mpg.select(
    mpg.hwy.alias("highway"),
    col("hwy").alias("highway"),
    expr("hwy").alias("highway"),
    expr("hwy AS highway"),
).show(5)

+-------+-------+-------+-------+
|highway|highway|highway|highway|
+-------+-------+-------+-------+
|     29|     29|     29|     29|
|     29|     29|     29|     29|
|     31|     31|     31|     31|
|     30|     30|     30|     30|
|     26|     26|     26|     26|
+-------+-------+-------+-------+
only showing top 5 rows



## Spark SQL

* Spark SQL allows us to write SQL queries against our spark dataframes
* We'll first "register" the table with staprk with `sp_df.createOrReplaceTempView("sp_df")`

In [29]:
mpg.createOrReplaceTempView("mpg")

In [30]:
# Now we can write a sql query against the `mpg` table

spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
)

DataFrame[hwy: bigint, cty: bigint, avg: double]

Notice that the resulting value is another dataframe. As we know, in order to view the values in a dataframe, we need to use `.show`

In [31]:
spark.sql(
    """
SELECT hwy, cty, (hwy + cty) / 2 AS avg
FROM mpg
"""
).show(5)

+---+---+----+
|hwy|cty| avg|
+---+---+----+
| 29| 18|23.5|
| 29| 21|25.0|
| 31| 20|25.5|
| 30| 21|25.5|
| 26| 16|21.0|
+---+---+----+
only showing top 5 rows



## Type Casting

View column datatypes:

In [32]:
mpg.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [33]:
mpg.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [34]:
# To convert the data types we use .cast

mpg.select(mpg.hwy.cast("string")).printSchema()

root
 |-- hwy: string (nullable = true)



## Basic Built-in Functions

There are many other functions beyond col and expr within the `pyspark.sql.functions` module for operating on pyspark dataframe columns. 

* `concat`: to concatenate string
* `sum`: to sum a group
* `avg`: to take the average of a group
* `min`: to find the minimun
* `max`: to find the max

**Note**: Importing the sum, min, and max functions directly will override the build-in sum, min and max functions. This means you will get an error if you try to sum a list of numbers, because sum will refrence the relative pyspark function, which works with pyspark dataframe columns, while the relative built-in function works with lists of numbers. 

In [35]:
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean

In [40]:
mpg.select(
    (sum(mpg.hwy) / count(mpg.hwy)).alias("average_1"),
    avg(mpg.hwy).alias("average_2"),
    min(mpg.hwy),
    max(mpg.hwy),
).show()

+-----------------+-----------------+--------+--------+
|        average_1|        average_2|min(hwy)|max(hwy)|
+-----------------+-----------------+--------+--------+
|23.44017094017094|23.44017094017094|      12|      44|
+-----------------+-----------------+--------+--------+



In [37]:
mpg.select(concat(mpg.manufacturer, mpg.model)).show(5)

+---------------------------+
|concat(manufacturer, model)|
+---------------------------+
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
+---------------------------+
only showing top 5 rows



In order to use a string literaly as part of our select, we'll need to use the `lit` function, otherwise spark will try to resolve our string as a column

In [41]:
from pyspark.sql.functions import lit
mpg.select(concat(mpg.cyl, lit(" cylinders"))).show(5)

+-----------------------+
|concat(cyl,  cylinders)|
+-----------------------+
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            4 cylinders|
|            6 cylinders|
+-----------------------+
only showing top 5 rows



## More `pyspark` Functions for String Manipulation

In [42]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [43]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410, San Antonio, TX 78227",
            ]
        }
    )
)

textdf.show(truncate=False)

+---------------------------------------------+
|address                                      |
+---------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215        |
|1255 SW Loop 410, San Antonio, TX 78227      |
+---------------------------------------------+



`regexp_extract`: Specifiy a regular expression with at least one capture group, and create a new column based on the contents of a capture group.

* FIrst argumetn: the name of the string column to extract from
* second argument: the regular expression itself
* last argument: specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it, and we wanted the contents of the 2nd group, we would sepecigy a 2 here. 

In [44]:
textdf.select(
    "address",
    regexp_extract("address", r"^(\d+)", 1).alias("street_no"),
    regexp_extract("address", r"^\d+\s([\w\s]+?),", 1).alias("street"),
).show(truncate=False)

+---------------------------------------------+---------+------------------+
|address                                      |street_no|street            |
+---------------------------------------------+---------+------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|600      |Navarro St ste 600|
|3130 Broadway St, San Antonio, TX 78209      |3130     |Broadway St       |
|303 Pearl Pkwy, San Antonio, TX 78215        |303      |Pearl Pkwy        |
|1255 SW Loop 410, San Antonio, TX 78227      |1255     |SW Loop 410       |
+---------------------------------------------+---------+------------------+



In the example above, the first argument to regexp_extract is the name of the string column to extract from, the second argument is the regular expression itself, and the last argument specifies which capture group we want to use. If, for example, our regular expression had 2 capture groups in it and we wanted the contents of the 2nd group, we would specify a 2 here.

In addition to `regexp_extract`, `regexp_replace` lets us make substitutions based on a regular expression.

In [45]:
textdf.select(
    "address",
    regexp_replace("address", r"^.*?,\s*", "").alias("city_state_zip"),
).show(truncate=False)

+---------------------------------------------+---------------------+
|address                                      |city_state_zip       |
+---------------------------------------------+---------------------+
|600 Navarro St ste 600, San Antonio, TX 78205|San Antonio, TX 78205|
|3130 Broadway St, San Antonio, TX 78209      |San Antonio, TX 78209|
|303 Pearl Pkwy, San Antonio, TX 78215        |San Antonio, TX 78215|
|1255 SW Loop 410, San Antonio, TX 78227      |San Antonio, TX 78227|
+---------------------------------------------+---------------------+



In our example above, we obtain just the city, state, and zip code of the address by replacing everything up to the first comma with an empty string.

## `.filter` and `.where`

Sprak provides two dataframe methods, `.filter` and `.where`, wich bot hallow us to select a subset of the rows of our dataframe.

In [46]:
mpg.filter(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()

+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|      model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+-----------+-----+----+---+----------+---+---+---+---+----------+
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 25| 32|  r|subcompact|
|       honda|      civic|  1.6|1999|  4|manual(m5)|  f| 23| 29|  p|subcompact|
|       honda|      civic|  1.6|1999|  4|  auto(l4)|  f| 24| 32|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|manual(m5)|  f| 26| 34|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
|       honda|      civic|  1.8|2008|  4|  auto(l5)|  f| 24| 36|  c|subcompact|
|       honda|      civic|  2.0|2008|  4|manual(m6)|  f| 21| 29|  p|subcompact|
|     hyundai|    tiburon|  2.0|1999|  4

## When and Otherwise

Similar to an `IF` in excel, `CASE...WHEN` in SQL, or `np.where` in python, sprak provides a `when` function. 

The `when` function lets us specify a condition, and a value to produce if that condition is true:

In [47]:
from pyspark.sql.functions import when

In [48]:
mpg.select(mpg.hwy, when(mpg.hwy > 25, "good_mileage").alias("mpg_desc")).show(
    12
)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25|        null|
| 28|good_mileage|
| 27|good_mileage|
| 25|        null|
+---+------------+
only showing top 12 rows



Notice here that if the condition we specified is false, `null` will be produced. Instead of null, we can specify a value to use if our condition is false with the `.otherwise` method.

In [49]:
mpg.select(
    mpg.hwy,
    when(mpg.hwy > 25, "good_mileage")
    .otherwise("bad_mileage")
    .alias("mpg_desc"),
).show(12)

+---+------------+
|hwy|    mpg_desc|
+---+------------+
| 29|good_mileage|
| 29|good_mileage|
| 31|good_mileage|
| 30|good_mileage|
| 26|good_mileage|
| 26|good_mileage|
| 27|good_mileage|
| 26|good_mileage|
| 25| bad_mileage|
| 28|good_mileage|
| 27|good_mileage|
| 25| bad_mileage|
+---+------------+
only showing top 12 rows



To specify multiple conditions, we can chain `.when` calls. The first condition that is met will be the value that is used, and if none of the conditions are met the value specified in the `.otherwise` will be used (or null if you don't provide a `.otherwise`).

In [50]:
mpg.select(
    mpg.displ,
    (
        when(mpg.displ < 2, "small")
        .when(mpg.displ < 3, "medium")
        .otherwise("large")
        .alias("engine_size")
    ),
).show(10)

+-----+-----------+
|displ|engine_size|
+-----+-----------+
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
|  2.0|     medium|
|  2.8|     medium|
|  2.8|     medium|
|  3.1|      large|
|  1.8|      small|
|  1.8|      small|
|  2.0|     medium|
+-----+-----------+
only showing top 10 rows



Notice here that a car with a `displ` of 1.8 matches both conditions we specified, but small is produced because it is associated with the first matching condition. For any value between 2 and 3, medium will be produced, and anything larger than 3 will produce large.

## Sorting and Ordering

Spark lets us sort the rows in our dataframe by one or multiple columns with two methods: `.sort`, and `.orderby`. `.sort` and `.orderby` are aliases of each other and do the exact same thing. Like other methods we've seen, `.sort` takes in a column object or a string that is the name of a column.

In [51]:
mpg.sort(mpg.hwy).show(8)

+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep| grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
|       dodge|        durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|  dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep| grand cherokee 4wd|  6.1|2008|  8|  auto(l5)|  4| 11| 14|  p|   suv|
|   chevrolet|    k1500 tahoe 4wd|  5.3|2008|  8|  auto(l4)|  4| 11| 14|  e|   suv|
|  land rover|        range rover|  4.0|1999|  8|  auto(l4)|  4| 11| 15|  p|   suv|
+------------+-------------------+-----+----+---+----------+---+---+---+---+

By default, values are sorted in ascending order. To sort in descending order, we can use the `.desc` method on any Column object, or the `desc`function from `pyspark.sql.functions.`

In [52]:
from pyspark.sql.functions import asc, desc

In [53]:
mpg.sort(mpg.hwy.desc())
# is the same as
mpg.sort(col("hwy").desc())
# is the same as
mpg.sort(desc("hwy")).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.8|2008|  4|  auto(l5)|  f| 25| 36|  r|subcompact|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



To specify sorting by multiple columns, we provide each column as a separate argument to `.sort`.

In [54]:
mpg.sort(desc("class"), mpg.cyl.asc(), col("hwy").desc()).show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-----+
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 20| 27|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 20| 26|  r|  suv|
|      subaru|      forester awd|  2.5|1999|  4|manual(m5)|  4| 18| 25|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|manual(m5)|  4| 19| 25|  p|  suv|
|      subaru|      forester awd|  2.5|1999|  4|  auto(l4)|  4| 18| 24|  r|  suv|
|      subaru|      forester awd|  2.5|2008|  4|  auto(l4)|  4| 18| 23|  p|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|manual(m5)|  4| 15| 20|  r|  suv|
|      toyota|       4runner 4wd|  2.7|1999|  4|  auto(l4)|  4| 16| 20|  r|  suv|
|        jeep|grand cherokee 4wd|  3.0|2008|  6|  auto(l5)|  4| 17| 22|  d|  suv|
|      nissan|  

## Grouping and Aggregating

To aggregate our data by group, we can use the `.groupBy` method. Like with `.select`, we can pass either Column objects or strings that are column names to `.groupBy`. All of the expressions below are equivalent.

In [55]:
mpg.groupBy(mpg.cyl)
mpg.groupBy(col("cyl"))
mpg.groupBy("cyl")

Once the data is grouped, we need to specify an aggregation. We can use one of the aggregate functions we imported earlier, alond with a column:

In [56]:
mpg.groupBy(mpg.cyl).agg(avg(mpg.cty), avg(mpg.hwy)).show()

+---+------------------+-----------------+
|cyl|          avg(cty)|         avg(hwy)|
+---+------------------+-----------------+
|  6| 16.21518987341772|22.82278481012658|
|  5|              20.5|            28.75|
|  8|12.571428571428571|17.62857142857143|
|  4|21.012345679012345|28.80246913580247|
+---+------------------+-----------------+



To group by multiple columns, pass each of the columns a a separate argument to `.groupBy` (Note that this is different from pandas, where we would need to pass a list).

In [57]:
mpg.groupBy("cyl", "class").agg(avg(mpg.cty), avg(mpg.hwy)).show()

+---+----------+------------------+------------------+
|cyl|     class|          avg(cty)|          avg(hwy)|
+---+----------+------------------+------------------+
|  5|   compact|              21.0|              29.0|
|  5|subcompact|              20.0|              28.5|
|  6|subcompact|              17.0|24.714285714285715|
|  6|    pickup|              14.5|              17.9|
|  4|subcompact|22.857142857142858| 30.80952380952381|
|  8|       suv|12.131578947368421|16.789473684210527|
|  8|    pickup|              11.8|              15.8|
|  8|   midsize|              16.0|              24.0|
|  4|   midsize|              20.5|           29.1875|
|  8|   2seater|              15.4|              24.8|
|  6|   compact|16.923076923076923|25.307692307692307|
|  6|   minivan|              15.6|              22.2|
|  4|   compact|            21.375|          29.46875|
|  8|subcompact|              14.8|              21.6|
|  6|   midsize|17.782608695652176| 26.26086956521739|
|  4|   mi

In addition to `.groupBy`, we can use `.rollup`, which will do the same aggregations, but will also include the overall total:

In [58]:
mpg.rollup("cyl").count().sort("cyl").show()

+----+-----+
| cyl|count|
+----+-----+
|null|  234|
|   4|   81|
|   5|    4|
|   6|   79|
|   8|   70|
+----+-----+



Here the null value in cyl indicates the total count.

In [59]:
mpg.rollup("cyl").agg(expr("avg(hwy)")).sort("cyl").show()

+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|null|23.44017094017094|
|   4|28.80246913580247|
|   5|            28.75|
|   6|22.82278481012658|
|   8|17.62857142857143|
+----+-----------------+



And in the example above, the null row represents the overall average highway mileage.

In [60]:
mpg.rollup("cyl", "class").mean("hwy").sort(col("cyl"), col("class")).show()

+----+----------+------------------+
| cyl|     class|          avg(hwy)|
+----+----------+------------------+
|null|      null| 23.44017094017094|
|   4|      null| 28.80246913580247|
|   4|   compact|          29.46875|
|   4|   midsize|           29.1875|
|   4|   minivan|              24.0|
|   4|    pickup|20.666666666666668|
|   4|subcompact| 30.80952380952381|
|   4|       suv|             23.75|
|   5|      null|             28.75|
|   5|   compact|              29.0|
|   5|subcompact|              28.5|
|   6|      null| 22.82278481012658|
|   6|   compact|25.307692307692307|
|   6|   midsize| 26.26086956521739|
|   6|   minivan|              22.2|
|   6|    pickup|              17.9|
|   6|subcompact|24.714285714285715|
|   6|       suv|              18.5|
|   8|      null| 17.62857142857143|
|   8|   2seater|              24.8|
+----+----------+------------------+
only showing top 20 rows



## Crosstabs and Pivot Tables

In addition to groupby, spark provides a couple other ways to do aggregation. One of which is `.crosstab`. This is very similary to pandas `.crosstab` function, in that it calculates the number of occurances of each unique value from the two passed columns:

In [61]:
mpg.crosstab("class", "cyl").show()

+----------+---+---+---+---+
| class_cyl|  4|  5|  6|  8|
+----------+---+---+---+---+
|   midsize| 16|  0| 23|  2|
|subcompact| 21|  2|  7|  5|
|   2seater|  0|  0|  0|  5|
|    pickup|  3|  0| 10| 20|
|   minivan|  1|  0| 10|  0|
|       suv|  8|  0| 16| 38|
|   compact| 32|  2| 13|  0|
+----------+---+---+---+---+



`.crosstab` simply does counts, if we want a different aggregation, we can use `.pivot`. For example, to find the average highway mileage for each combination of car class and number of cylinders, we could write the following:

In [62]:
mpg.groupby("class").pivot("cyl").mean("hwy").show()

+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   compact|          29.46875|29.0|25.307692307692307|              null|
|   minivan|              24.0|null|              22.2|              null|
|       suv|             23.75|null|              18.5|16.789473684210527|
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   2seater|              null|null|              null|              24.8|
+----------+------------------+----+------------------+------------------+



Here the unique values from the column we group by will be the rows in the resulting dataframe, and the unique values from the column we pivot on will become the columns. The values in each cell will be equal to the aggregation we specified over the group of values defined by the intersection of the rows and the columns.

## Handling Missing Data

Let's take a look at how sprak handles missing data. First we'll create a dataframe that has a few missing values:

In [63]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



Spark provides two main ways to deal with missing values:

* `.fill`: to replace missing values with a specified value
* `.drop`: to drop rows containing missing values


Both methods are accessed through the `.na` property. We'll look at some examples below:

In [64]:
df.na.drop().show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [65]:
df.na.fill(0).show()

+---+---+
|  x|  y|
+---+---+
|1.0|0.0|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|0.0|
+---+---+



For both methods, we can sepcify that we only want to fill or drop values in a specific column with a second argument:

In [66]:
df.na.fill(0, subset="x").show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|0.0|0.0|
|4.0|3.0|
|5.0|1.0|
|0.0|NaN|
+---+---+



Notice that above the na values in the x column were filled with 0, but the na values in y were left alone.

In [67]:
df.na.drop(subset="y").show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In the example above, the rows that had an na value for the y column were dropped, but the rows with na values for only the x column are still present.

## Explaining DataFrame Transformations

The `.explain` method will show us how spark is thinking about our dataframe.

In [68]:
mpg.explain()

== Physical Plan ==
Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


For our basic example, we see that there is only a single step.

In [69]:
mpg.select(mpg.cyl, mpg.hwy).explain()

== Physical Plan ==
*(1) Project [cyl#19L, hwy#23L]
+- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Notice now that there is another step after "Scan ExistingRDD", a "Project" that contains the names of the columns we are looking for.

In [70]:
mpg.select(((mpg.cyl + mpg.hwy) / 2).alias("avg_mpg")).explain()

== Physical Plan ==
*(1) Project [(cast((cyl#19L + hwy#23L) as double) / 2.0) AS avg_mpg#1377]
+- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Here we are doing a more advanced select calculation, but this is still just a single step to spark.

In [71]:
mpg.filter(mpg.cyl == 6).explain()

== Physical Plan ==
*(1) Filter (isnotnull(cyl#19L) && (cyl#19L = 6))
+- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Notice that our filter is also a single step.

Without reading ahead, do you think the execution plan for the two dataframes below will be the same or not?

In [72]:
mpg.select("cyl", "hwy").filter(expr("cyl = 6")).explain()
mpg.filter(expr("cyl = 6")).select("cyl", "hwy").explain()

== Physical Plan ==
*(1) Project [cyl#19L, hwy#23L]
+- *(1) Filter (isnotnull(cyl#19L) && (cyl#19L = 6))
   +- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]
== Physical Plan ==
*(1) Project [cyl#19L, hwy#23L]
+- *(1) Filter (isnotnull(cyl#19L) && (cyl#19L = 6))
   +- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Notice that even though we specified the transformations (`.select` and `.filter`) in a different order, we end up with the same output when we call `.explain`. This is because spark will look at our dataframe and transform it into the most efficient representation possible.

In [73]:
mpg.selectExpr("cyl + 3 * 16 / 4 + 19 AS unused", "hwy").select(
    "hwy"
).explain()

== Physical Plan ==
*(1) Project [hwy#23L]
+- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Notice here that we have 2 seperate select statements, but spark will condense this down to a single Project, as it is smart enough to realize that it doesn't actually need to do all the arithmetic we specified in the first select, since we arent using that value later on.

In [74]:
mpg.select(min(mpg.cyl)).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[min(cyl#19L)])
+- Exchange SinglePartition
   +- *(1) HashAggregate(keys=[], functions=[partial_min(cyl#19L)])
      +- *(1) Project [cyl#19L]
         +- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


Notice now that the execution plan gets much more complicated. This is because in steps prior, we were applying transformations that applied to each row individually. To calculate a minimum, we have to look at all the rows in the dataset to find the smallest.

In [75]:
mpg.groupby(mpg.cyl).agg(min(mpg.hwy), max(mpg.hwy)).explain()

== Physical Plan ==
*(2) HashAggregate(keys=[cyl#19L], functions=[min(hwy#23L), max(hwy#23L)])
+- Exchange hashpartitioning(cyl#19L, 200)
   +- *(1) HashAggregate(keys=[cyl#19L], functions=[partial_min(hwy#23L), partial_max(hwy#23L)])
      +- *(1) Project [cyl#19L, hwy#23L]
         +- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


In [76]:
(
    mpg.select(col("cyl"), expr("(cty + hwy) / 2 AS avg_mpg"))
    .filter(expr('class == "compact"'))
    .groupby("cyl")
    .agg(min("avg_mpg"), avg("avg_mpg"), max("avg_mpg"))
    .explain()
)

== Physical Plan ==
*(2) HashAggregate(keys=[cyl#19L], functions=[min(avg_mpg#1414), avg(avg_mpg#1414), max(avg_mpg#1414)])
+- Exchange hashpartitioning(cyl#19L, 200)
   +- *(1) HashAggregate(keys=[cyl#19L], functions=[partial_min(avg_mpg#1414), partial_avg(avg_mpg#1414), partial_max(avg_mpg#1414)])
      +- *(1) Project [cyl#19L, (cast((cty#22L + hwy#23L) as double) / 2.0) AS avg_mpg#1414]
         +- *(1) Filter (isnotnull(class#25) && (class#25 = compact))
            +- Scan ExistingRDD[manufacturer#15,model#16,displ#17,year#18L,cyl#19L,trans#20,drv#21,cty#22L,hwy#23L,fl#24,class#25]


## More Dataframe Manipulation Examples

Let's take a look at some more examples of working with spark dataframes. For these examples, we'll be working with a dataset of observations of the weather in seattle.

In [77]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



Let's print out the number of rows and columns in our dataset:

In [78]:
print(weather.count(), "rows", len(weather.columns), "columns")

1461 rows 6 columns


In [79]:
min_date, max_date = weather.select(min("date"), max("date")).first()
min_date, max_date

('2012-01-01', '2015-12-31')